## Notebook Objectives

1. Analyze total hospital charge distributions
2. Identify high-cost patients and diagnoses
3. Examine LOS–charge relationships
4. Analyze cost concentration (Pareto effects)
5. Generate financially actionable insights

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path

pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", "{:,.2f}".format)

sns.set_style("whitegrid")

In [ ]:
DATA_PATH = Path("../data/processed/hospital_inpatient_discharges_cleaned.csv")
df = pd.read_csv(DATA_PATH)

In [ ]:
df.head()

In [ ]:
df["total_charges"].describe()

In [ ]:
plt.figure(figsize=(8,4))
sns.histplot(np.log1p(df["total_charges"]), bins=40, kde=True)
plt.title("Log-Transformed Total Charges Distribution")
plt.xlabel("log(1 + Total Charges)")
plt.ylabel("Frequency")
plt.show()

In [ ]:
high_cost_threshold = df["total_charges"].quantile(0.95)
high_cost_threshold

In [ ]:
df["high_cost_case"] = df["total_charges"] >= high_cost_threshold
df["high_cost_case"].value_counts(normalize=True)

In [ ]:
charges_sorted = df.sort_values("total_charges", ascending=False)
charges_sorted["cumulative_cost_share"] = (
    charges_sorted["total_charges"].cumsum() /
    charges_sorted["total_charges"].sum()
)

charges_sorted.head()

In [ ]:
plt.figure(figsize=(6,5))
plt.plot(
    np.arange(1, len(charges_sorted) + 1) / len(charges_sorted),
    charges_sorted["cumulative_cost_share"]
)
plt.axhline(0.8, color="red", linestyle="--")
plt.xlabel("Proportion of Patients")
plt.ylabel("Cumulative Share of Charges")
plt.title("Cost Concentration (Pareto Curve)")
plt.show()

In [ ]:
sample_df = df.sample(n=min(5000, len(df)), random_state=42)

plt.figure(figsize=(6,5))
sns.scatterplot(
    data=sample_df,
    x="length_of_stay",
    y="total_charges",
    alpha=0.4
)
plt.title("Length of Stay vs Total Charges")
plt.xlabel("Length of Stay (Days)")
plt.ylabel("Total Charges")
plt.show()

In [ ]:
df["charges_per_day"] = df["total_charges"] / df["length_of_stay"]
df["charges_per_day"].describe()

In [ ]:
plt.figure(figsize=(8,4))
sns.histplot(np.log1p(df["charges_per_day"]), bins=40, kde=True)
plt.title("Log-Transformed Charges per Day")
plt.xlabel("log(1 + Charges per Day)")
plt.ylabel("Frequency")
plt.show()

In [ ]:
if "principal_diagnosis" in df.columns:
    top_dx = df["principal_diagnosis"].value_counts().head(10).index

In [ ]:
if "principal_diagnosis" in df.columns:
    dx_charges = (
        df[df["principal_diagnosis"].isin(top_dx)]
        .groupby("principal_diagnosis")["total_charges"]
        .mean()
        .sort_values(ascending=False)
    )
    dx_charges

In [ ]:
if "principal_diagnosis" in df.columns:
    plt.figure(figsize=(10,4))
    dx_charges.plot(kind="bar")
    plt.title("Average Charges by Top Diagnoses")
    plt.xlabel("Diagnosis")
    plt.ylabel("Average Charges")
    plt.show()

In [ ]:
if "hospital_name" in df.columns:
    hospital_charges = (
        df.groupby("hospital_name")
          .agg(
              avg_charges=("total_charges", "mean"),
              median_charges=("total_charges", "median"),
              discharges=("hospital_name", "count")
          )
          .query("discharges >= 100")
          .sort_values("avg_charges", ascending=False)
    )

    hospital_charges.head(10)

In [ ]:
numeric_cols = df.select_dtypes(include=["int64", "float64"]).columns
df[numeric_cols].corr()["total_charges"].sort_values(ascending=False)

In [ ]:
OUTPUT_DIR = Path("../outputs/tables")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

df[["total_charges", "length_of_stay", "charges_per_day"]].describe().to_csv(
    OUTPUT_DIR / "charges_summary_statistics.csv"
)

## Key Insights

* Hospital charges are **extremely right-skewed**
* ~5% of patients account for a disproportionate share of total charges
* LOS is the **single strongest driver** of total charges
* Certain diagnoses have **systematically higher daily costs**
* Hospital-level charge variation suggests pricing or case-mix differences